# Atmospheric River Map

 http://vortex.plymouth.edu/~j_cordeira/ARPortal/EU/Archive/2016121006/html/D1_IVT_0.html#tab
 The integrated vapor transport defines an AR. The region should be continous and $\ge$ 2000 km. The IVT $\ge$ 250 kg m$^{-1}$ s$^{-1}$.

http://journals.ametsoc.org/doi/abs/10.1175/MWR-D-13-00168.1
$$IVT = \frac{1}{g}\int_{p_sfc}^{100\,hPa} q \bf{V} dp$$


 http://apps.ecmwf.int/codes/grib/param-db
 #### Parameters from ECMWF
    - name = 'specific humidity',          shortName = 'q',     [kg kg^-1]
    - name = 'U component of wind',        shortName = 'u',      [m s^-1]
    - name = 'V component of wind',        shortName = 'v',      [m s^-1]



In [1]:
import pygrib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.basemap import shiftgrid
import math
from datetime import date
import calendar
import numpy as np
%matplotlib inline

### Plotting data on a map (Example Gallery) https://matplotlib.org/basemap/users/examples.html

In [2]:
### Define colorbar colors
champ = 255.
# tot. precipitable water (grey scale)
no1 = np.array([255,255,255])/champ
no2 = np.array([250,255,0])/champ
no3 = np.array([255,203,0])/champ
no4 = np.array([255,121,0])/champ
no5 = np.array([255,0,0])/champ
no6 = np.array([148,0,97])/champ
no7 = np.array([101,0,137])/champ

no8 = np.array([130,102,0])/champ




In [3]:
# Plotting data on a map (Example Gallery) https://matplotlib.org/basemap/users/examples.html
m = Basemap(projection='merc', \
            llcrnrlon=-80., urcrnrlon=50., \
            llcrnrlat=15.,urcrnrlat=75., \
            resolution='l')

#m = Basemap(projection='merc', \
 #           llcrnrlon=-180., urcrnrlon=180., \
  #          llcrnrlat=-75.,urcrnrlat=75., \
   #         resolution='l')

In [4]:
def opengrib(yyyy, mm, dd, tt, pm, path):
    grib = '%s/%s/param_%s_%s%s%s_%s00.grib' % (path,pm,pm,yyyy,mm,dd,tt)
    grbs = pygrib.open(grib)
    return(grbs);

In [5]:
def selectgrb(grbs, sN, tOL, lv):
    val = grbs.select()[0]
    val = grbs.select(shortName = sN, typeOfLevel = tOL, level = lv)[0]
    val = val.values
    return(val);

In [13]:
def shiftgrb(grb,val):
    lat,lon = grb.latlons()
    lons = lon[0,:]
    val,lons = shiftgrid(180., val, lons, start = False)
    lats = lat[:,0]
    
    lons,lats = np.meshgrid(lons,lats)
    plons,plats = m(lons,lats)
    return(plons,plats,val,lats);

In [7]:
def totalwind(wind_u, wind_v):
    V = np.sqrt(wind_u**2 + wind_v**2)
    theta = (180./math.pi) * np.arctan2(-wind_v,-wind_u)    # meteorological wind direction, [deg]
    return(V,theta);

In [8]:
# set the file name of your input GRIB file
year = '2016'
mon  = '12'
day = '10'     # day_time
time = '00'

path = '../test_dataECMWF/SCA/pl'

### specific humidity q
parameter = 'q'
grbsQ = opengrib(yyyy = year, mm = mon, dd = day, tt = time, pm = parameter, path = path)

### Wind
parameter = 'uv'
grbsUV = opengrib(yyyy = year, mm = mon, dd = day, tt = time, pm = parameter, path = path)


In [9]:
### Dates for plotting
yr = int(year)
mo = int(mon)
dy = int(day)
my_date = date(yr,mo,dy)
calday = calendar.day_name[my_date.weekday()]
calmon = calendar.month_abbr[mo]

In [10]:
grbQ = grbsQ.select()[0]            # finds the first grib message, if grbs.select(name='Maximum temperature') 
                                    # then first with matching name
grbUV = grbsUV.select()[0]           

In [11]:
### print an inventory of the file
# specific humidity
#grbsQ.seek(0)
#for grbQ in grbsQ:
#    print(grbQ)

In [15]:
### get the values into python
# find the first grib message with a matching name:

## specific humidity
# from Rutz et al. - 2014:
# The integration is done using data at the surface, 50-hPa intervals from the surface to 500 hPa,

# specific humidity q
SH = dict()

# Wind
wind_u = dict()       # U component of wind
wind_v = dict()       # V component of wind
V = dict()            # total wind vector
theta = dict()        # wind direction

for i in range(750,1000,50):       # ECMWF has only levels 1000-850 in 50hPa
    lv = i + 50
    SH[lv] = selectgrb(grbsQ, 'q', 'isobaricInhPa', lv) 
    wind_u[lv] = selectgrb(grbsUV, 'u', 'isobaricInhPa', lv)
    wind_v[lv] = selectgrb(grbsUV, 'v', 'isobaricInhPa', lv)
    ### Latitudes, Longitudes and shiftgrid
    # get the latitudes and longitudes of the grid
    plonsSH, platsSH, SH[lv],latsSH = shiftgrb(grbQ,SH[lv])
    plonsUV, platsUV, wind_u[lv],latsUV = shiftgrb(grbUV,wind_u[lv])
    plonsUV, platsUV, wind_v[lv],latsUV = shiftgrb(grbUV,wind_v[lv])
    V[lv], theta[lv] = totalwind(wind_u[lv],wind_v[lv])
    
for i in range(100,800,100):       # ECMWF has from 700 - 100 hPa only in 100hPa steps
    lv = i
    SH[lv] = selectgrb(grbsQ, 'q', 'isobaricInhPa', lv)
    wind_u[lv] = selectgrb(grbsUV, 'u', 'isobaricInhPa', lv)
    wind_v[lv] = selectgrb(grbsUV, 'v', 'isobaricInhPa', lv)
    ### Latitudes, Longitudes and shiftgrid
    # get the latitudes and longitudes of the grid
    plonsSH, platsSH, SH[lv],latsSH = shiftgrb(grbQ,SH[lv])
    plonsUV, platsUV, wind_u[lv],latsUV = shiftgrb(grbUV,wind_u[lv])
    plonsUV, platsUV, wind_v[lv],latsUV = shiftgrb(grbUV,wind_v[lv])
    V[lv], theta[lv] = totalwind(wind_u[lv],wind_v[lv])
    

In [16]:
## calculate the IVT
g = 9.81      # gravitational acceleration [m s^-2]


IVT = (1/g)*((SH[100]*V[100])*10000 + \
            (SH[200]*V[200])*10000 + \
            (SH[300]*V[300])*10000 + \
            (SH[400]*V[400])*10000 + \
            (SH[500]*V[500])*10000 + \
            (SH[600]*V[600])*10000 + \
            (SH[700]*V[700])*10000 + \
            (SH[800]*V[800])*10000 + \
            (SH[850]*V[850])*5000 + \
            (SH[900]*V[900])*5000 + \
            (SH[950]*V[950])*5000 +\
            (SH[1000]*V[1000])*5000
            )


print(IVT.max())

2238.64622603


In [17]:
x_IVT = dict()
y_IVT = dict()
for i in range(750,1000,50):       # ECMWF has only levels 1000-850 in 50hPa
    lv = i + 50
    x_IVT[lv] = - abs(IVT)* np.sin((math.pi/180)*theta[lv])
    y_IVT[lv] = - abs(IVT)* np.cos((math.pi/180)*theta[lv])

    
for i in range(100,800,100):       # ECMWF has from 700 - 100 hPa only in 100hPa steps
    lv = i
    x_IVT[lv] = - abs(IVT)* np.sin((math.pi/180)*theta[lv])
    y_IVT[lv] = - abs(IVT)* np.cos((math.pi/180)*theta[lv])



In [18]:
u_IVT = (x_IVT[100] +  \
        x_IVT[200] + \
        x_IVT[300] + \
        x_IVT[400] + \
        x_IVT[500] + \
        x_IVT[600] + \
        x_IVT[700] + \
        x_IVT[800] + \
        x_IVT[850] + \
        x_IVT[900] + \
        x_IVT[950] + \
        x_IVT[1000] 
        )

v_IVT = (y_IVT[100] +  \
        y_IVT[200] + \
        y_IVT[300] + \
        y_IVT[400] + \
        y_IVT[500] + \
        y_IVT[600] + \
        y_IVT[700] + \
        y_IVT[800] + \
        y_IVT[850] + \
        y_IVT[900] + \
        y_IVT[950] + \
        y_IVT[1000] 
        )



In [20]:
#uproj,vproj,xx,yy = m.transform_vector(u_IVT,v_IVT,plonsUV,latsUV,returnxy=True,masked=True)

TypeError: transform_vector() missing 2 required positional arguments: 'nx' and 'ny'

In [ ]:

### PLOT FIGURE
fig = plt.figure(figsize=(20,16))
ax = fig.add_subplot(1,1,1)
#
### Draw Latitude Lines
m.drawparallels(np.arange(-90.,120.,10.),labels=[1,0,0,0],fontsize=20,linewidth=0.2)
#
### Draw Longitude Lines
m.drawmeridians(np.arange(-180.,180.,10.),labels=[0,0,0,1],fontsize=20,linewidth=0.2)
#
### Draw Map
m.drawcoastlines()
m.drawmapboundary()
m.drawcountries()
m.fillcontinents(color='grey',alpha=0.1)


##
#
### Plot contour lines for IVT and fill
IVTlevels = np.arange(0,1900,250)
IVTmap = colors.ListedColormap([no1, no2, no3, no4, no5, no6, no7])
norm = colors.BoundaryNorm(boundaries = IVTlevels, ncolors=IVTmap.N)
cs = m.contourf(plonsUV, platsUV, IVT, IVTlevels, norm = norm, cmap=IVTmap)
CS2 = plt.contour(cs, levels=cs.levels,
                  linewidths=0.4,
                  colors=[no8])
### Plot direction and magnitude of IVT
#Q = m.quiver(xx,yy,uroj,vproj)


#
### Add Colorbar
cbaxes = fig.add_axes([0.14, 0.03, .75, .06] )   #[left, bottom, width, height] 
cbar = plt.colorbar(cs,orientation='horizontal',cax = cbaxes)#, cax = cbaxes)#, shrink=0.5)
cbar.ax.set_xlabel('IVT [kg$\,$m$^{-1}\,$s$^{-1}$]',fontsize='22')
cbar.ax.tick_params(labelsize=20) 
# Make a colorbar for the ContourSet returned by the contourf call.
# Add the contour line levels to the colorbar
cbar.add_lines(CS2)

#cbar.ax.set_aspect(200)
# Add arrows to show the IVT vectors


#
### Add Textbox
ax.text(0.98,0.94, '%s, %s %s %s   %s$\,$UTC' %(calday, day, calmon, year, time),     # x, y
            verticalalignment = 'bottom',  horizontalalignment='right',
            transform = ax.transAxes,
            color ='blue', fontsize=26,
            bbox={'facecolor':'white','alpha':1., 'pad':10})
#
### Add Title
#fig.suptitle('Integrated Vapor Transport', fontsize=20, fontweight='bold') 
#ax.set_title('MSLP (solid contours every 4$\,$hPa), \n 250$\,$hPa wind speed (m$\,$s$^{-1}$, shaded according to the color bar), \n 1000-500$\,$hPa thickness (dashed contours every 6 dam), \n total precipitable water (mm, shaded according to gray scale)',             fontsize=13)

### Save
#plt.savefig('../synoptic_figs/Atm_Riv/%s%s%s_%s.png' % (year, mon, day,time))
## with header
#plt.savefig('../synoptic_figs/Jet_Thick_SLP/%s%s%s_%s_header.png' % (year, mon, day,time))

In [ ]:
plt.close()